In [ ]:
import autogis

In [ ]:
import osmnx as ox
from pprint import pprint
import pandas as pd

In [ ]:
# load the geocordinates of the study areas
areas = autogis.GeoPoint(file='data/study_areas.csv')
geodata = areas.geodata
graphs = [f"data/{'_'.join(n.split(' '))}.graphml" for n in geodata.place]
geodata

#### for each study area extract the statistical indicators from the downloaded street network graph

In [ ]:
df = None
for city, place, graph in zip(geodata.city, geodata.place, graphs):
    G = ox.load_graphml(graph)
    rname = f"{city}-{place}"
    bstats = ox.basic_stats(ox.project_graph(G),
                            area=autogis.get_graph_area(G),
                            clean_intersects=True,
                            circuity_dist='euclidean')
    for k, count in bstats['streets_per_node_counts'].items():
        bstats[f"{k}way_count"] = count
    for k, prop in bstats['streets_per_node_proportions'].items():
        bstats[f"{k}way_proportion"] = prop
    del bstats['streets_per_node_counts']
    del bstats['streets_per_node_proportions']
    
    estats = ox.extended_stats(G, ecc=True, bc=True, cc=True)
    stat = estats.copy()
    for key, val in estats.items():
        if type(val) is dict:
            del stat[key]
    
    if df is None:
        df = pd.DataFrame(pd.Series(bstats|stat, name=rname)).round(3)
        continue
    ndf = pd.DataFrame(pd.Series(bstats|stat, name=rname)).round(3)
    df = pd.concat([df, ndf], axis=1)
    
df = df.fillna(0)
df.to_csv("data/network_indicators.csv")
df